In [17]:
import pandas as pd
import pystan

In [7]:
data = pd.read_csv("dat_lag.csv", index_col=0)

In [14]:
data_stan = {
    "x1": data.x1,
    "x2": data.x2,
    "y": data.y,
    "n": data.shape[0]
}

In [19]:
model_code = """
data {
    int n;
    vector[n] x1;
    vector[n] x2;
    vector[n] y;
}
parameters {
    real<lower=0> a1;
    real<lower=0> a2;
    real<lower=0, upper=1> lag;
    real<lower=0> sigma;
}
transformed parameters {
    vector<lower=0>[n] mu;
    
    mu[1] = a1*x1[1] + a2*x2[1];
    for(i in 2:n)
        mu[i] = a1*x1[i] + a2*x2[i] + a2*lag*x2[i-1];
}
model {
    y ~ normal(mu, sigma);
}
"""

In [20]:
model = pystan.StanModel(model_code=model_code)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_17592478f68fe51847b299a4f34b0bd4 NOW.


In [24]:
stanfit = model.sampling(data=data_stan)

In [25]:
stanfit

Inference for Stan model: anon_model_17592478f68fe51847b299a4f34b0bd4.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

         mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
a1        0.5  4.6e-9 2.6e-7    0.5    0.5    0.5    0.5    0.5   3276    1.0
a2        0.4  5.6e-9 2.8e-7    0.4    0.4    0.4    0.4    0.4   2505    1.0
lag       0.5  2.1e-8 1.0e-6    0.5    0.5    0.5    0.5    0.5   2345    1.0
sigma  5.8e-4  1.6e-4 2.5e-4 2.6e-4 3.6e-4 5.6e-4 7.4e-4 1.1e-3      2   3.43
mu[1]   700.0  5.3e-6 2.8e-4  700.0  700.0  700.0  700.0  700.0   2697    1.0
mu[2]   100.0  3.4e-6 1.7e-4  100.0  100.0  100.0  100.0  100.0   2460    1.0
mu[3]   400.0  5.6e-6 2.8e-4  400.0  400.0  400.0  400.0  400.0   2505    1.0
mu[4]  1600.0  7.5e-6 4.3e-4 1600.0 1600.0 1600.0 1600.0 1600.0   3210    1.0
mu[5]  1200.0  8.0e-6 4.5e-4 1200.0 1200.0 1200.0 1200.0 1200.0   3216    1.0
mu[6]   400.0  5.6e-6 2.8e-4  4